In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.firefox.options import Options
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
browser = Chrome() #open chrome in selenium
browser.get('https://meduza.io/specials/native') # open page with a lot of urls to ads

In [3]:
list_ = []
links = browser.find_elements_by_xpath('//*[@href]') # find all urls on page
for ii in links:
    print(ii.get_attribute('href')) 
    list_.append(ii.get_attribute('href')) #append all hrefs to a list

https://meduza.io/manifest.json
https://meduza.io/rss/all
https://meduza.io/rss/news
https://meduza.io/rss/fun
https://meduza.io/image/attachments/images/000/014/758/original/UO7gsH-WIDAM-OjCbP34Hw.png
https://meduza.io/specials/native
https://meduza.io/assets/images/safari-icons/apple-touch-icon-1024.png
https://meduza.io/assets/images/safari-icons/apple-touch-icon-76.png
https://meduza.io/assets/images/safari-icons/apple-touch-icon-120.png
https://meduza.io/assets/images/safari-icons/apple-touch-icon-76@2x.png
https://meduza.io/assets/images/safari-icons/apple-touch-icon-180.png
https://meduza.io/image/attachments/images/000/017/651/original/qAvIE71BRQOTl9q96t0kRA.svg
https://meduza.io/assets/main.e8f3d0fcc4ff731f8442fd0f3c675ed5.css
https://adservice.google.ru/adsid/integrator.js?domain=meduza.io
https://adservice.google.com/adsid/integrator.js?domain=meduza.io
https://meduza.io/
https://meduza.io/
https://meduza.io/articles
https://meduza.io/razbor
https://meduza.io/games
https://m

https://meduza.io/games/kakoy-vy-predprinimatel
https://meduza.io/games/naskolko-akkuratno-vy-vodite-avtomobil
https://meduza.io/quiz/u-vas-peresadka-v-aeroportu-helsinki-smozhete-provesti-vremya-s-polzoy
https://meduza.io/slides/postich-agile-osvoit-scrum
https://meduza.io/games/smozhete-li-ispolzovat-svoy-smartfon-na-100
https://meduza.io/cards/chto-takoe-postgarantiynoe-obsluzhivanie-i-zachem-mne-eto-nuzhno
https://meduza.io/cards/mne-nravyatsya-podkasty-na-meduze-mozhno-li-v-nih-kupit-reklamu-spoyler-da
https://meduza.io/feature/2018/03/06/stydnye-voprosy-o-semeynom-byudzhete
https://meduza.io/games/kakoy-podarok-budet-dlya-vas-idealnym
https://meduza.io/slides/pochemu-rukodelie-eto-kruto-i-ego-pora-perestat-stesnyatsya
https://meduza.io/quiz/merenge-ili-koko-loko
https://meduza.io/slides/9-situatsiy-kogda-vam-prosto-neobhodim-vysokoskorostnoy-internet
https://meduza.io/quiz/znaete-li-vy-kak-lyudi-raznyh-epoh-pytalis-sohranit-molodost
https://meduza.io/slides/desyat-veschey-kotorye

https://meduza.io/quiz/karmin-kadmievyy-ili-kinovar
https://meduza.io/quiz/zachem-nuzhen-etot-predmet
https://meduza.io/games/naskolko-vy-zhertva-marketinga
https://meduza.io/cards/u-menya-skoro-budet-rebenok-kak-podgotovit-dom-k-ego-poyavleniyu
https://meduza.io/slides/kak-splanirovat-menyu-na-nedelyu
https://meduza.io/games/uznayte-gde-by-vy-zhili-v-gorode-svoey-mechty
https://meduza.io/cards/kak-pravilno-myt-golovu
https://meduza.io/slides/10-situatsiy-kogda-bez-brancha-ne-oboytis
https://meduza.io/games/skazhite-chto-vy-lyubite-i-my-pridumaem-kuda-vam-poehat-na-elektrichke
https://meduza.io/quiz/igraem-v-assotsiatsii-ugadayte-pesnyu-po-naboru-kartinok
https://meduza.io/quiz/umnyy-dom-eto-mozhet-ili-poka-esche-net
https://meduza.io/games/kuhnya-kakoy-strany-vam-podhodit
https://meduza.io/video/2017/11/23/9-5-chasov-sborki-lego-technic-za-2-minuty
https://meduza.io/quiz/ponimaete-li-vy-o-chem-poyut-zarubezhnye-pop-zvezdy
https://meduza.io/quiz/v-klub-ili-na-ring-dlya-chego-eti-krosso

https://meduza.io/feature/2017/07/26/za-chto-my-ne-lyubim-seo
https://meduza.io/games/tartar-ili-pastrami-ugadayte-blyudo-po-ingredientam
https://meduza.io/cards/hochu-kupit-samokat-kakoy-vybrat
https://meduza.io/games/vzletit-ili-ne-vzletit
https://meduza.io/cards/posle-trenirovki-ochen-bolyat-myshtsy-chto-delat
https://meduza.io/cards/pochemu-tak-mnogo-lyudey-rabotayut-ne-po-spetsialnosti
https://meduza.io/cards/chernaya-pyatnitsa-na-rynke-nedvizhimosti-chto
https://meduza.io/slides/9-sposobov-otlichit-originalnyy-produkt-ot-poddelki
https://meduza.io/quiz/kino-golubi-i-gondoly-chto-vy-znaete-o-razvlecheniyah-venetsii
https://meduza.io/cards/kak-organizovat-transfer-v-lyubom-gorode-mira-i-ne-soyti-s-uma
https://meduza.io/feature/2017/07/14/vashey-sleduyuschey-kameroy-budet-bezzerkalka
https://meduza.io/cards/u-menya-slomalas-mashina-kak-izbezhat-ada-s-avtoservisami
https://meduza.io/slides/zachem-idti-na-festival-lastochka
https://meduza.io/feature/2017/07/12/kak-uvelichit-slovarnyy-

https://meduza.io/cards/kognitivnye-servisy-zachem-oni-nam
https://meduza.io/feature/2017/02/09/koshelki-ryukzaki-i-monetnitsa
https://meduza.io/cards/moemu-rebenku-nuzhno-podgotovitsya-k-ege-kak-eto-sdelat
https://meduza.io/cards/internet-veschey-chto-eto-takoe-i-kak-eto-rabotaet
https://meduza.io/quiz/chto-vy-ponimaete-v-skidkah
https://meduza.io/feature/2017/01/27/stoit-li-otpravlyat-rebenka-uchitsya-za-granitsu
https://meduza.io/quiz/zapomnit-vse
https://meduza.io/games/viber-zabanit-ili-promolchat
https://meduza.io/games/tele-2-chto-v-ostatke
https://meduza.io/quiz/klinika-atlas-vy-k-immunologu-ili-otorinolaringologu
https://meduza.io/quiz/azbuka-attikus-krasnaya-komnata-ili-chernyy-vigvam
https://meduza.io/cards/lingualeo-mne-nuzhno-vyuchit-angliyskiy-chto-delat
https://meduza.io/cards/neyropaticheskaya-bol-chto-eto-voobsche-takoe-u-menya-ona-est
https://meduza.io/feature/2017/01/08/kak-my-zhili-do-ayfonov
https://meduza.io/cards/kak-vybrat-torgovyy-tsentr-dlya-novogodnih-pokupok

https://meduza.io/cards/chto-sluchilos-s-amurskim-tigrom
https://meduza.io/quiz/giroskop-ili-akselerometr
https://meduza.io/cards/ya-postoyanno-sizhu-v-telefone-kak-s-etim-byt
https://meduza.io/feature/2016/06/23/anatomiya-bashni
https://meduza.io/feature/2016/06/22/zhizn-vzaymy
https://meduza.io/games/goroda-booking
https://meduza.io/games/platit-ili-ne-platit
https://meduza.io/feature/2016/06/16/valyutnyy-krizis-sanktsii-i-deshevaya-neft
https://meduza.io/feature/2016/06/14/pryamaya-liniya-s-uzhasom
https://meduza.io/feature/2016/06/11/ekonomika-2000-h-ekonomicheskiy-rost-i-delo-yukosa
https://meduza.io/quiz/vy-pochuyali-pachuli
https://meduza.io/cards/dlya-chego-nuzhno-mba
https://meduza.io/cards/ya-muzykant-kak-mne-proslavitsya
https://meduza.io/games/vypustyat-li-vas-za-granitsu
https://meduza.io/feature/2016/06/06/1998-god-glubokiy-no-kratkiy-shok
https://meduza.io/cards/chto-proishodit-kogda-vy-rasplachivaetes-kartoy
https://meduza.io/cards/ya-zadolzhal-dengi-chto-mne-za-eto-bud

https://meduza.io/pages/about
https://meduza.io/specials/native#auth
https://meduza.io/undefined


In [4]:
meduza_ads = []
for item in list_:
    if item[:23] == 'https://meduza.io/cards' or item[:25] == 'https://meduza.io/feature': # get rid of non-ad urls
        meduza_ads.append(item)
meduza_ads_ = meduza_ads[1:-5] # get rid of not important urls like: how meduza's using coockie, etc

In [6]:
len(meduza_ads_) # check is len of meduza_ads long enough

386

In [7]:
p = [] #create list
df = pd.DataFrame() #create df
for item in meduza_ads_:
    r = requests.get(str(item)) #open url 
    soup = BeautifulSoup(r.text, 'html5lib') # get html as text
    doc = {} #create dictionary
    if item[:23] == 'https://meduza.io/cards': # get only values that firsst values equals to https://meduza.io/cards
        doc['text'] = soup.find('div', {'class' : 'CardMaterial-body'}).text #get text from card url
        text = doc['text']
    elif not soup.find('div', {'class' : 'Lead'}):
        continue
    else:
        doc['text'] = soup.find('div', {'class' : 'Lead'}).text #get text from feature url
        doc['txt'] = soup.find('div', {'class' : 'Body'}).text #get text from feature url
        text = ''.join((doc['text'], doc['txt'])) # join 2 string to 1
    p.append(text) #every iterate we append all texts to list
    df = pd.DataFrame.from_records(({"value": i} for i in p)) #convert list to dataframe

In [8]:
df.to_csv('parser_ad.csv', index=False) #save df to csv

In [9]:
a = pd.read_csv('parser_ad.csv') # check csv
a 

,value
0,1.Что случилось?2.От чего конкретно страхует к...
1,1.Что это вообще такое — остеопороз?2.Откуда о...
2,1.Что случилось?2.А разве государственной пенс...
3,\n Династия Мин правила Китаем 300 ...
4,"\n Увы, некоторые состояния и особе..."
5,"1.Вступление2.Итак, я хочу развивать свой бизн..."
6,1.Что случилось?2.Что такое карта для бизнеса?...
7,\n Чуть меньше половины россиян име...
8,1.Что случилось?2.Какие плюсы дает работа по ф...
9,1.Весной я меняю резину с зимней на летнюю. Ещ...
